In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

In [ ]:
import holodeck.pop_observational

PATH_DATA = holo._PATH_DATA

FNAME_CONTINUOUS_DATA = "mbhb-pops-continuous_casey-clyde_mingarelli_2021-02-17.npz"
FNAME_MCCONNELL_MA_2013 = "mcconnell+ma-2013_1211.2816.txt"

FNAME_DATA = os.path.join(PATH_DATA, FNAME_CONTINUOUS_DATA)
FNAME_MCCONNELL_MA_2013 = os.path.join(PATH_DATA, FNAME_MCCONNELL_MA_2013)

# Quick: Population, Evolution, GW Spectrum

In [ ]:
# ---- Create initial population
#### pop = holo.population.BP_Continuous(FNAME_DATA)
pop = holo.pop_observational.BP_Observational()

In [ ]:
# ---- Evolve binary population
time_fixed = 2.0 * GYR
fixed = holo.evolution.Fixed_Time.from_pop(pop, time_fixed)
evo = holo.evolution.Evolution(pop, fixed)
evo.evolve()

In [ ]:
# # ---- Calculate and Plot GWB

# freqs = holo.utils.nyquist_freqs(dur=20.0*YR, cad=0.3*YR)
# gwb = holo.gravwaves.GW_Continuous(evo, freqs, nreals=30)
# plot.plot_gwb(gwb)
# plt.show()

In [ ]:
freqs = holo.utils.nyquist_freqs(20.0*YR, 0.3*YR)
gwb = holo.gravwaves.GW_Continuous(evo, freqs)
gwb.emit()
strain = gwb.strain

xx = freqs * YR
# print(f"{xx.shape=}, {strain.shape=}")
fig, ax = plt.subplots(figsize=[10, 6])
ax.set(xscale='log', xlabel='Frequency [1/yr]', yscale='log', ylabel='Characteristic Strain')
ax.plot(xx, strain)

# tw = ax.twinx(); tw.set(yscale='log')
# _draw_plaw(tw, freqs, f0=1, color='k', lw=3.0, alpha=0.5)
# _twin_hz(ax, nano=True, fs=12)

plt.show()